In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, normalize
import torch, os, random, copy
import numpy as np
import gc
from torch.nn.utils import clip_grad_norm_
from modules.ogb.graph_aug import mask_nodes, mask_edges, permute_edges, drop_nodes, subgraph
from torch import nn
from torch.nn import functional as F
# from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from torch_ema import ExponentialMovingAverage
from modules.ogb.utils import smiles2graph
from modules.dualgraph.mol import smiles2graphwithface, simles2graphwithface_with_mask
from modules.dualgraph.gnn import one_hot_atoms, one_hot_bonds, GNN2
from rdkit.Chem import AllChem
from rdkit import Chem
from torch_geometric.data import Dataset, InMemoryDataset
from modules.dualgraph.dataset import DGData

from torch_geometric.loader import DataLoader
from rdkit.Chem import PandasTools

In [98]:
import os
from itertools import repeat
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import subgraph, to_networkx

class MoleculeDataset_graphcl(InMemoryDataset):
    def __init__(self,
                 root='dataset_path',
                 transform=None,
                 pre_transform=None, 
                 df=None):
        self.df = df
        self.aug_prob = None
        self.aug_mode = 'sample'
        self.aug_strength = 0.2
        self.augmentations = [self.node_drop, self.subgraph,
                              self.edge_pert, self.attr_mask, lambda x: x]
        super().__init__(root, transform, pre_transform, df)
        
    @property
    def raw_file_names(self):
        return [f'raw_{i+1}.pt' for i in range(self.df.shape[0])]

    @property
    def processed_file_names(self):
        return [f'data_{i+1}.pt' for i in range(self.df.shape[0])]
    
    def set_augMode(self, aug_mode):
        self.aug_mode = aug_mode

    def set_augStrength(self, aug_strength):
        self.aug_strength = aug_strength

    def set_augProb(self, aug_prob):
        self.aug_prob = aug_prob

    def node_drop(self, data):
        node_num, _ = data.x.size()
        _, edge_num = data.edge_index.size()
        drop_num = int(node_num * self.aug_strength)

        idx_perm = np.random.permutation(node_num)
        idx_nodrop = idx_perm[drop_num:].tolist()
        idx_nodrop.sort()

        node_mask = torch.zeros(node_num).bool()
        node_mask[~torch.tensor(idx_nodrop)]=True

        edge_idx, edge_attr, edge_mask = subgraph(subset=idx_nodrop,
                                        edge_index=data.edge_index,
                                        edge_attr=data.edge_attr,
                                        relabel_nodes=True,
                                        num_nodes=node_num,
                                        return_edge_mask=True)
        data = self.ring_drop(data, node_mask, edge_mask)
        
        data.edge_index = edge_idx
        data.edge_attr = edge_attr
        data.x = data.x[idx_nodrop]        

        data.__num_nodes__, _ = data.x.shape
        return data

    def edge_pert(self, data):        
        node_num, _ = data.x.size()
        _, edge_num = data.edge_index.size()
        pert_num = int(edge_num * self.aug_strength)

        # delete edges
        idx_drop = np.random.choice(edge_num, (edge_num - pert_num),
                                    replace=False)
        edge_index = data.edge_index[:, idx_drop]
        edge_attr = data.edge_attr[idx_drop]

        # add edges
        adj = torch.ones((node_num, node_num))
        adj[edge_index[0], edge_index[1]] = 0
        # edge_index_nonexist = adj.nonzero(as_tuple=False).t()
        edge_index_nonexist = torch.nonzero(adj, as_tuple=False).t()
        idx_add = np.random.choice(edge_index_nonexist.shape[1],
                                    pert_num, replace=False)
        edge_index_add = edge_index_nonexist[:, idx_add]
        # random 4-class & 3-class edge_attr for 1st & 2nd dimension
        edge_attr_add_1 = torch.tensor(np.random.randint(
            4, size=(edge_index_add.shape[1], 1)))
        edge_attr_add_2 = torch.tensor(np.random.randint(
            3, size=(edge_index_add.shape[1], 1)))
        edge_attr_add_3 = torch.tensor(np.random.randint(
            2, size=(edge_index_add.shape[1], 1)))
        edge_attr_add = torch.cat((edge_attr_add_1, edge_attr_add_2, edge_attr_add_3), dim=1)
        edge_index = torch.cat((edge_index, edge_index_add), dim=1)
        

        edge_attr = torch.cat((edge_attr, edge_attr_add), dim=0)

        data.edge_index = edge_index
        data.edge_attr = edge_attr
        return data

    def attr_mask(self, data):

        _x = data.x.clone()
        node_num, _ = data.x.size()
        mask_num = int(node_num * self.aug_strength)

        token = data.x.float().mean(dim=0).long()
        idx_mask = np.random.choice(
            node_num, mask_num, replace=False)

        _x[idx_mask] = token
        data.x = _x
        return data    
    
    def ring_drop(self, graph, node_mask, edge_mask):
        node_mask_idx = (node_mask.cumsum(0) - 1) * node_mask
        edge_mask_idx = (edge_mask.cumsum(0) - 1) * edge_mask

        graph.ring_index = graph.ring_index[:, edge_mask]

        graph.n_edges = edge_mask.sum().item()
        graph.n_nodes = node_mask.sum().item()

        nf_node_mask = ~torch.isin(graph.nf_node, torch.where(~node_mask)[0])[0]
        # for ridx in range(graph.num_rings):        
        #     ring_node_mask = nf_node_mask[graph.nf_ring[0] == ridx+1]
        #     if not ring_node_mask.all():
        #         graph.ring_mask[ridx] = False

        graph.nf_node = node_mask_idx[graph.nf_node[:, nf_node_mask]]
        graph.nf_ring = graph.nf_ring[:, nf_node_mask]
        graph.n_nfs = graph.nf_node.size(1)
        return graph
    
    def subgraph(self, data):

        G = to_networkx(data)
        node_num, _ = data.x.size()
        _, edge_num = data.edge_index.size()
        sub_num = int(node_num * (1 - self.aug_strength))

        idx_sub = [np.random.randint(node_num, size=1)[0]]
        idx_neigh = set([n for n in G.neighbors(idx_sub[-1])])

        while len(idx_sub) <= sub_num:
            if len(idx_neigh) == 0:
                idx_unsub = list(set([n for n in range(node_num)]).difference(set(idx_sub)))
                idx_neigh = set([np.random.choice(idx_unsub)])
            sample_node = np.random.choice(list(idx_neigh))

            idx_sub.append(sample_node)
            idx_neigh = idx_neigh.union(
                set([n for n in G.neighbors(idx_sub[-1])])).difference(set(idx_sub))

        idx_nondrop = idx_sub
        idx_nondrop.sort()
        
        node_mask = torch.zeros(node_num).bool()
        node_mask[~torch.tensor(idx_nondrop)]=True

        edge_idx, edge_attr, edge_mask = subgraph(subset=idx_nondrop,
                                       edge_index=data.edge_index,
                                       edge_attr=data.edge_attr,
                                       relabel_nodes=True,
                                       num_nodes=node_num,
                                       return_edge_mask=True)
        data = self.ring_drop(data, node_mask, edge_mask)

        data.edge_index = edge_idx
        data.edge_attr = edge_attr
        data.x = data.x[idx_nondrop]
        data.__num_nodes__, _ = data.x.shape
        return data

    def get(self, idx):
        sid = self.sid_list[idx]        
        data = torch.load(f'graph_pt/{sid}_addh.pt')
        data1 = copy.deepcopy(data)
        data2 = copy.deepcopy(data)

        if self.aug_mode == 'no_aug':
            n_aug1, n_aug2 = 4, 4
            data1 = self.augmentations[n_aug1](data1)
            data2 = self.augmentations[n_aug2](data2)
        elif self.aug_mode == 'uniform':
            n_aug = np.random.choice(25, 1)[0]
            n_aug1, n_aug2 = n_aug // 5, n_aug % 5
            data1 = self.augmentations[n_aug1](data1)
            data2 = self.augmentations[n_aug2](data2)
        elif self.aug_mode == 'sample':
            n_aug = np.random.choice(25, 1, p=self.aug_prob)[0]
            n_aug1, n_aug2 = n_aug // 5, n_aug % 5
            data1 = self.augmentations[n_aug1](data1)
            data2 = self.augmentations[n_aug2](data2)
        else:
            raise ValueError
        
        return data, data1, data2
    
    def process(self):
        self.sid_list = []        
        
        for i in range(self.df.shape[0]):
            sid = self.df.loc[i, 'ID']            
            self.sid_list.append(sid)
    def len(self):
        return self.df.shape[0]            

In [99]:
data = PandasTools.LoadSDF('/home/pjh/CODE/mol/Site-of-metabolism/Site-of-metabolism/data/train_0611.sdf')
data['ID'] = 'TRAIN' + data.index.astype(str).str.zfill(4)

In [100]:
graph = torch.load('graph_pt/TRAIN0000_addh.pt')

In [102]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

    torch.cuda.manual_seed(seed)  # type: ignore
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
seed_everything(2023)


In [105]:
train_dataset = MoleculeDataset_graphcl(df = data)

Processing...
Done!


In [106]:
train_dataset.set_augMode('sample')
train_dataset.set_augProb(np.ones(25) / 25)
train_dataset.set_augStrength(0.2)

In [107]:
# batch1.ring_mask
# batch1.nf_node
# batch1.nf_ring

In [108]:
triplet_loss = nn.TripletMarginLoss(margin=0.0, p=2)
criterion = nn.CrossEntropyLoss()

In [109]:
class GraphContrastiveLearning(torch.nn.Module):

    def __init__(self):
        super(GraphContrastiveLearning, self).__init__()
        self.ddi = True
        self.gnn = GNN2(
                        mlp_hidden_size = 512,
                        mlp_layers = 2,
                        num_message_passing_steps=8,
                        latent_size = 128,
                        use_layer_norm = True,
                        use_bn=False,
                        use_face=True,
                        som_mode=False,
                        ddi=True,
                        dropedge_rate = 0.1,
                        dropnode_rate = 0.1,
                        dropout = 0.1,
                        dropnet = 0.1,
                        global_reducer = 'sum',
                        node_reducer = 'sum',
                        face_reducer = 'sum',
                        graph_pooling = 'sum',                        
                        node_attn = True,
                        face_attn = True,
                        encoder_dropout=0.0,                        
                        )
                        
        self.proj = nn.Sequential(
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        )
    def forward(self, batch):
        mol = self.gnn(batch).squeeze(1)
        return self.proj(mol)

In [110]:
device = 'cuda:0'
epochs = 100
lr = 1e-5
weight_decay = 5e-5
batch_size= 256

In [111]:
train_dataset = MoleculeDataset_graphcl(df = data)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers = 0)

model = GraphContrastiveLearning().to(device)

optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)
ema = ExponentialMovingAverage(model.parameters(), decay=0.999)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=epochs, verbose=False)

Processing...
Done!


In [112]:
for batch, batch1, batch2 in tqdm(train_loader):
    with torch.no_grad():
        out =  model(batch.to(device))
        out1 = model(batch1.to(device))
        out2 = model(batch2.to(device))

 27%|██▋       | 67/246 [00:08<00:22,  8.13it/s]

In [ ]:
def loss_cl(x1, x2):
    T = 0.1
    batch_size, _ = x1.size()
    x1_abs = x1.norm(dim=1)
    x2_abs = x2.norm(dim=1)

    sim_matrix = torch.einsum('ik,jk->ij', x1, x2) / torch.einsum('i,j->ij', x1_abs, x2_abs)
    sim_matrix = torch.exp(sim_matrix / T)
    pos_sim = sim_matrix[range(batch_size), range(batch_size)]
    loss = pos_sim / (sim_matrix.sum(dim=1) - pos_sim)
    loss = - torch.log(loss).mean()
    return loss

In [ ]:
state_dict = torch.load(f'/home/pjh/workspace/SOM/graph_mamba/ckpt_pretrain/gnn_pretrain_epoch36.pt')

model.load_state_dict(state_dict['model_state_dict'])
scheduler.load_state_dict(state_dict['scheduler_state_dict'])
ema.load_state_dict(state_dict['ema_state_dict'] )
optim.load_state_dict(state_dict['optimizer_state_dict'])

In [ ]:
best_val_loss = 1e6
start = 1
for epoch in range(37, epochs+1):

    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        batch = [bat.to(device) for bat in batch]
        outputs = [model(bat) for  bat in batch]        
        origin_output = outputs[0]
        
        mask_cl_loss = loss_cl(outputs[1], outputs[2])                    
        mask_t_loss = triplet_loss(outputs[0], outputs[1], outputs[2])

        loss = mask_cl_loss + (mask_t_loss * 0.1)

        optim.zero_grad()
        loss.backward() 
        optim.step()
        ema.update()
        
        train_loss += loss.cpu().item()
        
        gc.collect()
        torch.cuda.empty_cache()

    if train_loss < best_val_loss:
        best_val_loss = train_loss
        torch.save(model.gnn.state_dict(), f'ckpt_pretrain/gnn_pretrain.pt')
        

    scheduler.step()
    torch.save(
            {
            'optimizer_state_dict': optim.state_dict(),
            'model_state_dict': model.state_dict(),
            'gnn_state_dict' : model.gnn.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'ema_state_dict' : ema.state_dict()
            },
            f'ckpt_pretrain/gnn_pretrain_epoch{epoch}.pt')

    print(f'EPOCH : {epoch} | train_loss : {train_loss/len(train_loader):.4f}')



In [ ]:
# EPOCH : 1 | train_loss : -2.3468
# EPOCH : 2 | train_loss : -3.3075
# EPOCH : 3 | train_loss : -3.5078
# EPOCH : 4 | train_loss : -3.6116
# EPOCH : 5 | train_loss : -3.6767
# EPOCH : 6 | train_loss : -3.7220
# EPOCH : 7 | train_loss : -3.7547
# EPOCH : 8 | train_loss : -3.7785
# EPOCH : 9 | train_loss : -3.7976
# EPOCH : 10 | train_loss : -3.8130
# EPOCH : 11 | train_loss : -3.8248
# EPOCH : 12 | train_loss : -3.8358
# EPOCH : 13 | train_loss : -3.8443
# EPOCH : 14 | train_loss : -3.8521
# EPOCH : 15 | train_loss : -3.8593
# EPOCH : 16 | train_loss : -3.8657
# EPOCH : 17 | train_loss : -3.8720
# EPOCH : 18 | train_loss : -3.8774
# EPOCH : 19 | train_loss : -3.8831
# EPOCH : 20 | train_loss : -3.8879
# EPOCH : 21 | train_loss : -3.8916
# EPOCH : 22 | train_loss : -3.8959
# EPOCH : 23 | train_loss : -3.8989
# EPOCH : 24 | train_loss : -3.9027
# EPOCH : 25 | train_loss : -3.9056
# EPOCH : 26 | train_loss : -3.9086
# EPOCH : 27 | train_loss : -3.9118
# EPOCH : 28 | train_loss : -3.9144
# EPOCH : 29 | train_loss : -3.9176
# EPOCH : 30 | train_loss : -3.9199
# EPOCH : 31 | train_loss : -3.9221
# EPOCH : 32 | train_loss : -3.9248
# EPOCH : 33 | train_loss : -3.9270
# EPOCH : 34 | train_loss : -3.9289
# EPOCH : 35 | train_loss : -3.9310
# EPOCH : 36 | train_loss : -3.9328
# EPOCH : 37 | train_loss : -3.9343

In [ ]:
# add H
# 100%|██████████| 10113/10113 [2:39:32<00:00,  1.06it/s] 
# EPOCH : 1 | train_loss : -2.2254
# 100%|██████████| 10113/10113 [2:40:06<00:00,  1.05it/s] 
# EPOCH : 2 | train_loss : -3.2680
# 100%|██████████| 10113/10113 [2:40:23<00:00,  1.05it/s] 
# EPOCH : 3 | train_loss : -3.4840
# 100%|██████████| 10113/10113 [2:40:28<00:00,  1.05it/s] 
# EPOCH : 4 | train_loss : -3.5963
# 100%|██████████| 10113/10113 [2:40:39<00:00,  1.05it/s] 
# EPOCH : 5 | train_loss : -3.6643
# 100%|██████████| 10113/10113 [2:41:22<00:00,  1.04it/s] 
# EPOCH : 5 | train_loss : -3.6644
# 100%|██████████| 10113/10113 [2:43:59<00:00,  1.03it/s] 
# EPOCH : 6 | train_loss : -3.7121
# 100%|██████████| 10113/10113 [2:43:19<00:00,  1.03it/s] 
# EPOCH : 7 | train_loss : -3.7456
# 100%|██████████| 10113/10113 [2:43:16<00:00,  1.03it/s] 
# EPOCH : 8 | train_loss : -3.7727
# 100%|██████████| 10113/10113 [2:42:57<00:00,  1.03it/s] 
# EPOCH : 9 | train_loss : -3.7932
# 100%|██████████| 10113/10113 [2:41:36<00:00,  1.04it/s] 
# EPOCH : 10 | train_loss : -3.8113
# 100%|██████████| 10113/10113 [2:42:50<00:00,  1.04it/s] 
# EPOCH : 11 | train_loss : -3.8263
# 100%|██████████| 10113/10113 [2:42:50<00:00,  1.04it/s] 
# EPOCH : 12 | train_loss : -3.8376
# 100%|██████████| 10113/10113 [2:43:01<00:00,  1.03it/s] 
# EPOCH : 13 | train_loss : -3.8482
# 100%|██████████| 10113/10113 [2:43:19<00:00,  1.03it/s] 
# EPOCH : 14 | train_loss : -3.8564
# 100%|██████████| 10113/10113 [2:42:09<00:00,  1.04it/s] 
# EPOCH : 15 | train_loss : -3.8636
# 100%|██████████| 10113/10113 [2:42:25<00:00,  1.04it/s] 
# EPOCH : 16 | train_loss : -3.8719
# 100%|██████████| 10113/10113 [2:42:51<00:00,  1.03it/s] 
# EPOCH : 17 | train_loss : -3.8782
# 100%|██████████| 10113/10113 [2:42:59<00:00,  1.03it/s] 
# EPOCH : 18 | train_loss : -3.8842
# 100%|██████████| 10113/10113 [2:43:17<00:00,  1.03it/s] 
# EPOCH : 19 | train_loss : -3.8888
# 100%|██████████| 10113/10113 [2:43:05<00:00,  1.03it/s] 
# EPOCH : 20 | train_loss : -3.8924

In [ ]:
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | CYP          |   jac_bond |   f1s_bond |   apc_bond | n_bond     |   jac_spn |   f1s_spn |   apc_spn | n_spn    |   jac_som |   f1s_som |   apc_som | n_som      |
# +==============+============+============+============+============+===========+===========+===========+==========+===========+===========+===========+============+
# | BOM_1A2      |     0.1907 |     0.3203 |     0.2288 | 106 / 5772 |    0.1429 |    0.2500 |    0.2953 | 5 / 399  |    0.1892 |    0.3182 |    0.2252 | 111 / 6171 |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2A6      |     0.1791 |     0.3038 |     0.2654 | 40 / 5772  |    0.0000 |    0.0000 |    0.0776 | 3 / 399  |    0.1714 |    0.2927 |    0.2501 | 43 / 6171  |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2B6      |     0.0828 |     0.1529 |     0.0952 | 49 / 5772  |    0.0000 |    0.0000 |    0.2849 | 5 / 399  |    0.0802 |    0.1486 |    0.0925 | 54 / 6171  |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2C8      |     0.1257 |     0.2233 |     0.1366 | 91 / 5772  |    0.0000 |    0.0000 |    0.0174 | 3 / 399  |    0.1237 |    0.2202 |    0.1327 | 94 / 6171  |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2C9      |     0.1125 |     0.2022 |     0.1489 | 82 / 5772  |    0.2500 |    0.4000 |    0.3667 | 7 / 399  |    0.1169 |    0.2094 |    0.1457 | 89 / 6171  |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2C19     |     0.1354 |     0.2385 |     0.1494 | 79 / 5772  |    0.0000 |    0.0000 |    0.4628 | 7 / 399  |    0.1314 |    0.2322 |    0.1469 | 86 / 6171  |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2D6      |     0.2328 |     0.3776 |     0.2657 | 133 / 5772 |    0.0000 |    0.0000 |    0.3995 | 8 / 399  |    0.2250 |    0.3673 |    0.2633 | 141 / 6171 |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_2E1      |     0.2537 |     0.4048 |     0.3449 | 47 / 5772  |    0.0000 |    0.0000 |    0.1081 | 4 / 399  |    0.2394 |    0.3864 |    0.3265 | 51 / 6171  |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | BOM_3A4      |     0.2350 |     0.3805 |     0.3263 | 276 / 5772 |    0.0667 |    0.1250 |    0.2614 | 14 / 399 |    0.2303 |    0.3744 |    0.3191 | 290 / 6171 |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# | CYP_REACTION |     0.3712 |     0.5415 |     0.5213 | 417 / 5772 |    0.2400 |    0.3871 |    0.2820 | 19 / 399 |    0.3663 |    0.5362 |    0.5111 | 436 / 6171 |
# +--------------+------------+------------+------------+------------+-----------+-----------+-----------+----------+-----------+-----------+-----------+------------+
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | CYP          |   jac_hdx |   f1s_hdx |   apc_hdx | n_hdx      |   jac_oxi |   f1s_oxi |   apc_oxi | n_oxi      |   jac_clv |   f1s_clv |   apc_clv | n_clv      |   jac_rdc |   f1s_rdc |   apc_rdc | n_rdc     |
# +==============+===========+===========+===========+============+===========+===========+===========+============+===========+===========+===========+============+===========+===========+===========+===========+
# | BOM_1A2      |    0.2097 |    0.3467 |    0.2576 | 56 / 1995  |    0.1750 |    0.2979 |    0.2319 | 43 / 5772  |    0.2024 |    0.3366 |    0.2826 | 37 / 3777  |    0.0000 |    0.0000 |    0.1736 | 2 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2A6      |    0.2286 |    0.3721 |    0.3068 | 17 / 1995  |    0.1724 |    0.2941 |    0.2553 | 19 / 5772  |    0.0000 |    0.0000 |    0.2840 | 21 / 3777  |    0.0000 |    0.0000 |   -0.0000 | 0 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2B6      |    0.1154 |    0.2069 |    0.1252 | 24 / 1995  |    0.0333 |    0.0645 |    0.0859 | 22 / 5772  |    0.0328 |    0.0635 |    0.1410 | 17 / 3777  |    0.0000 |    0.0000 |   -0.0000 | 0 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2C8      |    0.1240 |    0.2206 |    0.1306 | 59 / 1995  |    0.1286 |    0.2278 |    0.1572 | 31 / 5772  |    0.1194 |    0.2133 |    0.2064 | 26 / 3777  |    0.0000 |    0.0000 |   -0.0000 | 0 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2C9      |    0.0919 |    0.1683 |    0.1598 | 46 / 1995  |    0.0886 |    0.1628 |    0.1611 | 31 / 5772  |    0.1571 |    0.2716 |    0.2537 | 23 / 3777  |    0.0000 |    0.0000 |    0.0497 | 3 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2C19     |    0.1083 |    0.1954 |    0.1559 | 43 / 1995  |    0.1538 |    0.2667 |    0.1508 | 31 / 5772  |    0.1733 |    0.2955 |    0.2411 | 27 / 3777  |    0.0000 |    0.0000 |    0.2429 | 2 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2D6      |    0.1731 |    0.2951 |    0.2224 | 77 / 1995  |    0.2778 |    0.4348 |    0.2831 | 48 / 5772  |    0.3500 |    0.5185 |    0.3478 | 45 / 3777  |    0.0000 |    0.0000 |    0.1949 | 3 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_2E1      |    0.2059 |    0.3415 |    0.3988 | 21 / 1995  |    0.2500 |    0.4000 |    0.2891 | 24 / 5772  |    0.0000 |    0.0000 |    0.1906 | 13 / 3777  |    0.0000 |    0.0000 |    0.2429 | 2 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | BOM_3A4      |    0.2104 |    0.3476 |    0.3099 | 146 / 1995 |    0.2690 |    0.4240 |    0.3440 | 112 / 5772 |    0.3121 |    0.4757 |    0.3667 | 97 / 3777  |    0.0000 |    0.0000 |    0.1316 | 9 / 3777  |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+
# | CYP_REACTION |    0.3341 |    0.5009 |    0.4879 | 229 / 1995 |    0.4167 |    0.5882 |    0.5132 | 162 / 5772 |    0.4355 |    0.6067 |    0.5908 | 147 / 3777 |    0.0000 |    0.0000 |    0.1445 | 11 / 3777 |
# +--------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+